## Passo 1 - Importar Arquivos e Bibliotecas

In [2]:
#Importanto bibliotecas
import pandas as pd

In [ ]:
#Importando bases de dados

emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

display(emails)
display(lojas)
display(vendas)

## Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [9]:
#Incluir nome das lojas em vendas
vendas = vendas.merge(lojas, on='ID Loja')
display(vendas)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


In [12]:
# Criando um dicionário que permite o armazenamento de uma tabela de vendas para cada loja existente

dicionario_lojas = {}
for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]

display(dicionario_lojas['Salvador Shopping'])

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
96969,81,2019-01-02,8,Cueca Xadrez,1,60,60,Salvador Shopping
96970,126,2019-01-02,8,Camisa Estampa,1,113,113,Salvador Shopping
96971,138,2019-01-02,8,Pulseira Linho,5,114,570,Salvador Shopping
96972,177,2019-01-02,8,Camisa Liso,2,105,210,Salvador Shopping
96973,194,2019-01-02,8,Camisa Gola V Linho,4,145,580,Salvador Shopping
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


In [14]:
dia_indicador = vendas['Data'].max()

print(dia_indicador)
print('{}/{}'.format(dia_indicador.day, dia_indicador.month))

2019-12-26 00:00:00
26/12


## Passo 3 - Salvar planilha na pasta de backup

In [16]:
# Identificar se a pasta já existe
import pathlib

caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]


for arquivo in arquivos_pasta_backup:
    lista_nomes_backup.append(arquivo.name)


for loja in dicionario_lojas:
    #Criando uma pasta para cada loja na lista de nomes
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()


    # Salvando dentro da pasta de backup
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo

    dicionario_lojas[loja].to_excel(local_arquivo)


## Passo 4 - Calcular indicador para 1 loja

In [27]:
loja = "Norte Shopping"
vendas_loja = dicionario_lojas[loja]
vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

# Faturamento
faturamento_ano = vendas_loja['Valor Final'].sum()
faturamento_dia = vendas_loja_dia['Valor Final'].sum()

print("Faturamento: ano", faturamento_ano," /  dia", faturamento_dia)

# Diversidade de produtos
qtde_produtos_ano = len(vendas_loja['Produto'].unique())
qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

print("Diversidade de Produtos: ano", qtde_produtos_ano," /  dia", qtde_produtos_dia)

# Ticket medio
valor_venda = vendas_loja.groupby('Código Venda').sum()
ticket_medio_ano = valor_venda['Valor Final'].mean()

valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
ticket_medio_ano - valor_venda_dia['Valor Final'].mean()

print("Ticket Médio: ano", ticket_medio_ano," /  dia", ticket_medio_ano)


Faturamento: ano 1711968  /  dia 1259
Diversidade de Produtos: ano 120  /  dia 2
Ticket Médio: ano 784.5866177818515  /  dia 784.5866177818515
